# 形状変更

データの形状を変更する操作は、データ分析や加工において重要なステップです。Polarsでは効率的に形状を変更するための様々な方法を提供しています。この章では、その中でも特に重要な操作について説明します。

In [7]:
import polars as pl
from helper.jupyter import row

## ピボットテーブル

`pivot`メソッドを使用すると、データフレームをピボットテーブルの形式に変換できます。この操作では、指定した列を基準にしてデータを集計し、行列形式で再構築します。

以下の例では、`name`列を新しい列ラベルとして使用し、`index`列を行ラベル（インデックス）として設定しています。また、`value`列の値を新しいテーブルのセルに挿入し、同じ行と列の組み合わせが複数存在する場合には`aggregate_function`で指定した方法（この場合は`first`）で値を集約します。


In [8]:
df = pl.DataFrame(
    dict(
        index=[0, 1, 2, 0, 1, 2, 0, 1, 2, 1],
        name=['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'A'],
        value=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        score=[1, 3, 5, 4, 3, 2, 1, 2, 3, 4],
    )
)

df2 = df.pivot('name', index='index', values='value', aggregate_function='first')
row(df, df2)

index,name,value,score
i64,str,i64,i64
0,"""A""",10,1
1,"""A""",20,3
2,"""A""",30,5
0,"""B""",40,4
1,"""B""",50,3
2,"""B""",60,2
0,"""C""",70,1
1,"""C""",80,2
2,"""C""",90,3


`aggregate_function`引数を使用して、同じ行列の組み合わせに対応する複数の値をどのように集約するかを指定できます。Polarsはこの引数に以下のような標準的な関数を受け付けます：

1. **`sum`**: 値を合計します。
2. **`mean`**: 値の平均を計算します。
3. **カスタム演算式**: ユーザー定義の集約演算式を適用可能です。ここでは`pl.element()`ですべての値をリストに集約します。

以下の例を通して、異なる`aggregate_function`を使用したピボットの結果を確認します。

In [9]:
df3 = df.pivot('name', index='index', values='value', aggregate_function='sum')
df4 = df.pivot('name', index='index', values='value', aggregate_function='mean')
df5 = df.pivot('name', index='index', values='value', aggregate_function=pl.element())
row(df3, df4, df5)

index,A,B,C
i64,i64,i64,i64
0,10,40,70
1,120,50,80
2,30,60,90
index,A,B,C
i64,f64,f64,f64
0,10.0,40.0,70.0
1,60.0,50.0,80.0
2,30.0,60.0,90.0
index,A,B,C


`values`引数に複数の列を渡すことで、指定した複数の列がピボットテーブルの中で展開されます。この場合、結果のテーブルでは、列ラベルが`name`列の値に基づき、各ラベルに対応するサブ列として指定された値の列が表示されます。

In [10]:
df.pivot('name', index='index', values=['value', 'score'], aggregate_function=pl.element())

index,value_A,value_B,value_C,score_A,score_B,score_C
i64,list[i64],list[i64],list[i64],list[i64],list[i64],list[i64]
0,[10],[40],[70],[1],[4],[1]
1,"[20, 100]",[50],[80],"[3, 4]",[3],[2]
2,[30],[60],[90],[5],[2],[3]


`separator`引数を指定することで、列名と値の結合文字を自由に変更できます。

In [11]:
df.pivot('name', index='index', values=['value', 'score'], aggregate_function="max", separator=":")

index,value:A,value:B,value:C,score:A,score:B,score:C
i64,i64,i64,i64,i64,i64,i64
0,10,40,70,1,4,1
1,100,50,80,4,3,2
2,30,60,90,5,2,3


## TODO

In [26]:
def unpivot_by_prefix(df, column_name_sep, group_name="group"):
    sep = column_name_sep
    groups = [name.split(sep)[0] for name in df.columns]
    dfs = []
    for group in dict.fromkeys(groups):
        index = [i for i, name in enumerate(groups) if name == group]
        dfs.append(
            df.select(
                pl.lit(group).alias(group_name),
                pl.nth(index).name.map(lambda name:name.split(sep)[-1]))
        )
    
    df_long = pl.concat(dfs)
    return df_long

In [28]:
unpivot_by_prefix(wide_df.select(pl.exclude("index")), ":")

group,A,B,C
str,i64,i64,i64
"""value""",10,40,70
"""value""",100,50,80
"""value""",30,60,90
"""score""",1,4,1
"""score""",4,3,2
"""score""",5,2,3


In [21]:
from polars import selectors as cs

In [22]:
wide_df.unpivot(on=cs.ends_with("A"), index="index")

index,variable,value
i64,str,i64
0,"""value_A""",10
1,"""value_A""",100
2,"""value_A""",30
0,"""score_A""",1
1,"""score_A""",4
2,"""score_A""",5


`unstack()`はNumPyのreshapeと似ています。一つの列を複数の列に分けます。

In [10]:
long_df[:-1].unstack(3, how='vertical', columns='value')

value_0,value_1,value_2
i64,i64,i64
10,40,70
20,50,80
30,60,90


In [11]:
long_df[:-1].unstack(3, how='horizontal', columns=['value'])

value_0,value_1,value_2
i64,i64,i64
10,20,30
40,50,60
70,80,90
